# 다중회귀

## 음악 재생 수 예측하기
- 사용자의 음악 스트리밍 데이터를 기반으로 특정 곡의 재생 횟수를 예측하는 다중 회귀 모델을 만들고 평가해보자
- 제공 데이터
    - BPM (Beats Per Minute): 곡의 속도
    - Energy: 에너지 지표 (0~100)
    - Danceability: 춤출 수 있는 정도 (0~100)
    - Length: 곡의 길이 (초 단위)
    - Popularity: 곡의 인기 점수 (0~100)
    - Play_Count: 해당 곡의 총 재생 횟수 (목표 변수)

### 데이터 로드 및 탐색
- MusicStreaming.csv 데이터를 불러오고 기본적인 통계를 확인해보기
- 파일명 : high_popularity_spotify_data.csv
- 결측값이 있는지 확인하고, 필요하면 처리

In [1]:
import pandas as pd

df = pd.read_csv('../data/high_popularity_spotify_data.csv')
df[['track_popularity']]

,track_popularity
0,100
1,97
2,93
3,81
4,98
...,...
1681,76
1682,74
1683,69
1684,82


In [2]:
df.isnull().sum()

energy                      0
tempo                       0
danceability                0
playlist_genre              0
loudness                    0
liveness                    0
valence                     0
track_artist                0
time_signature              0
speechiness                 0
track_popularity            0
track_href                  0
uri                         0
track_album_name            1
playlist_name               0
analysis_url                0
track_id                    0
track_name                  0
track_album_release_date    0
instrumentalness            0
track_album_id              0
mode                        0
key                         0
duration_ms                 0
acousticness                0
id                          0
playlist_subgenre           0
type                        0
playlist_id                 0
dtype: int64

In [3]:
df = df.dropna()

### 데이터 전처리
- Genre를 원-핫 인코딩하여 숫자로 변환하기
- StandardScaler를 사용하여 BPM, Energy, Danceability, Length, Popularity를 정규화하기
- train_test_split을 사용하여 훈련 데이터와 테스트 데이터로 나누기

In [ ]:
df[['playlist_genre']].value_counts()

In [8]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# 데이터 로드
df = pd.read_csv('../data/high_popularity_spotify_data.csv')

# 컬럼명 정리 (공백 제거)
df.columns = df.columns.str.strip()

# 숫자형 데이터만 추출 (문자열 컬럼 자동 제거)
df_numeric = df.select_dtypes(include=[np.number])

# track_popularity를 target으로 설정
song_target = df_numeric[['track_popularity']]

# track_popularity를 제외한 입력 데이터 (숫자형만 사용)
song_input = df_numeric.drop(columns=['track_popularity'])

# PCA 적용 (5개 주요 특성 추출)
pca = PCA(n_components=5)
song_input_pca = pca.fit_transform(song_input)

song_input_pca

array([[ 3.71058737e+04,  3.74415221e+01,  6.11983398e-01,
        -1.28886217e+00, -6.19975768e-01],
       [-4.18912549e+03, -1.61913481e+01, -3.32369817e+00,
        -3.33296694e+00,  3.55736869e-01],
       [-4.82621256e+04, -1.32324560e+01, -4.36298142e+00,
         2.53012200e+00,  3.71057678e-01],
       ...,
       [-2.05221254e+04, -2.23554065e+01,  7.19635838e-01,
         1.80533928e+00, -5.40072147e-01],
       [-2.46691252e+04, -3.80424739e+01, -4.25364077e+00,
         2.47169929e+00,  3.93932507e-01],
       [-9.47512583e+03,  5.68483462e+00,  6.32233741e-01,
         1.64840759e+00,  4.36768033e-01]], shape=(1686, 5))

In [ ]:
from sklearn.model_selection import train_test_split
song_input = df[['tempo', 'energy', 'danceability', 'duration_ms']]
song_target = df[['track_popularity']]
train_input, test_input, train_target, test_target = train_test_split(song_input, song_target)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### 다중 회귀 모델 훈련
- `LinearRegression`을 사용하여 모델을 훈련시키기
- `track_popularity`를 예측하는 회귀 모델을 만들고, `R²` 점수를 평가하기

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target)

print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

### 새로운 곡의 재생 수 예측
- 임의의 새로운 곡 데이터를 입력하고, 모델이 재생 횟수를 예측하도록 해보기

In [ ]:
from sklearn.linear_model import SGDRegressor
sc = SGDRegressor(loss='squared_error', max_iter=100)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

In [ ]:
sc.partial_fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

### 결과 시각화
- 예측된 track_popularity와 실제 값을 비교하는 **산점도(scatter plot)**를 그리기

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(test_target, lr.predict(test_poly))
plt.xlabel('Actual Popularity')
plt.xlabel('Predicted Popularity')